# Apple - Pro Content Creator Mac Software Usage Insights

```SQL
CREATE TABLE fct_multimedia_usage (
    user_id integer,
    usage_date date,
    hours_spent decimal
);

INSERT INTO fct_multimedia_usage (user_id, usage_date, hours_spent)
VALUES
    (101, '2024-07-31', 2.3),
    (102, '2024-07-31', 3.0),
    (103, '2024-07-31', 1.5),
    (101, '2024-07-30', 1.0),
    (104, '2024-07-29', 4.0),
    (105, '2024-07-31', 2.0),
    (101, '2024-08-01', 3.4),
    (102, '2024-08-01', 2.6),
    (103, '2024-08-02', 5.0),
    (104, '2024-08-03', 1.0),
    (102, '2024-08-05', 4.5),
    (105, '2024-08-05', 3.5),
    (106, '2024-08-07', 2.0),
    (107, '2024-08-07', 6.0),
    (101, '2024-09-01', 3.0),
    (102, '2024-09-01', 2.0),
    (103, '2024-09-02', 1.5),
    (104, '2024-09-02', 3.5),
    (105, '2024-09-03', 4.0),
    (106, '2024-09-03', 2.0),
    (107, '2024-09-03', 1.0),
    (108, '2024-09-04', 7.0),
    (109, '2024-09-04', 2.0),
    (101, '2024-09-04', 1.0),
    (110, '2024-09-05', 5.0),
    (111, '2024-09-05', 4.0),
    (112, '2024-09-05', 3.0);

SELECT * FROM fct_multimedia_usage;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_multimedia = pd.read_csv('Data/014/fct_multimedia_usage.csv')

df_multimedia.head()

,user_id,usage_date,hours_spent
0,101,2024-07-31,2.3
1,102,2024-07-31,3.0
2,103,2024-07-31,1.5
3,101,2024-07-30,1.0
4,104,2024-07-29,4.0


# Pregunta 1

### Como Analista de Producto en el equipo de software para Mac, necesitas entender el compromiso (engagement) de los creadores de contenido profesional con las herramientas multimedia. ¿Cuál es el número de usuarios distintos en el último día de julio de 2024?

In [5]:
df_july = df_multimedia[(df_multimedia['usage_date'] == '2024-07-31')]

resultado = df_july['user_id'].nunique()

resultado

4

```SQL
SELECT
    COUNT(DISTINCT user_id) AS distinct_users
FROM fct_multimedia_usage
WHERE usage_date = '2024-07-31'
```

# Pregunta 2

### Como Analista de Producto en el equipo de software para Mac, estás evaluando cuánto tiempo pasan los creadores de contenido profesional usando las herramientas multimedia. ¿Cuál es el número promedio de horas dedicadas por los usuarios durante agosto de 2024? Redondea el resultado hacia arriba al número entero más cercano.

In [8]:
df_august = df_multimedia[(df_multimedia['usage_date'].between('2024-08-01','2024-08-31'))]

respuesta2 = int(np.ceil(df_august['hours_spent'].mean()))

respuesta2

4

```SQL
SELECT
    CEIL(AVG(hours_spent)) AS average_hours_spent
FROM fct_multimedia_usage
WHERE usage_date BETWEEN '2024-08-01' AND '2024-08-31';
```

# Pregunta 3

### Como Analista de Producto en el equipo de software para Mac, estás investigando patrones de uso diario excepcionales en septiembre de 2024. Para cada día, determina el recuento de usuarios distintos y el total de horas dedicadas al uso de herramientas multimedia. ¿Qué días tienen ambas métricas por encima de sus respectivos valores promedio diarios de septiembre de 2024?

In [12]:
df_multimedia['usage_date'] = pd.to_datetime(df_multimedia['usage_date'])

df_sep = df_multimedia[df_multimedia['usage_date'].dt.month == 9]
daily_metrics = df_sep.groupby('usage_date').agg(
    daily_users = ('user_id','nunique'),
    daily_hours = ('hours_spent', 'sum')
)

avg_users = daily_metrics['daily_users'].mean()
avg_hours = daily_metrics['daily_hours'].mean()

dias_top = daily_metrics[
    (daily_metrics['daily_users'] > avg_users) &
    (daily_metrics['daily_hours'] > avg_hours)
]

dias_top

,daily_users,daily_hours
usage_date,,
2024-09-04,3,10.0
2024-09-05,3,12.0


```SQL
WITH daily_stats AS (
    SELECT
        usage_date,
        COUNT(DISTINCT user_id) AS daily_users,
        SUM(hours_spent) AS daily_hours
    FROM fct_multimedia_usage
    WHERE usage_date BETWEEN '2024-09-01' AND '2024-09-30'
    GROUP BY usage_date
)
SELECT usage_date
FROM daily_stats
WHERE daily_users > (SELECT AVG(daily_users) FROM daily_stats)
AND daily_hours > (SELECT AVG(daily_hours) FROM daily_stats);
```